In [18]:
import re
import string
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd

In [19]:
doc_1 = "Espresso é o armazenamento de dados distribuído consistente com a linha do tempo orientado a documentos do LinkedIn. Os clientes que buscam dados de perfil leem um subconjunto de campos de documentos inteiros do Espresso em diferentes casos de uso. Existem três partes principais do Espresso: roteadores, nós de armazenamento e gerenciamento de cluster. Os roteadores recebem solicitações de clientes online e encaminham as solicitações aos nós de armazenamento. Rotas que especificam o URI para obter o nome do banco de dados e procuram na tabela de roteamento os nós de armazenamento apropriados para os quais enviar solicitações. Ele também reúne os resultados e os envia de volta aos clientes. Um nó de armazenamento armazena e fornece documentos; fornece suporte a transações para documentos com a mesma chave de partição; e mantém o log de confirmação para replicação. O Espresso usa Apache Helix para monitorar e manter o cluster."
doc_2 = "Você terá um cliente acessando um sistema com acesso à várias bases dados e terá sistemas que estarão alimentado estas bases e fazendo sua gestão."

corpus = np.array([doc_1, doc_2])

### 1) Aplicar lowercase e remover caracteres

In [20]:
def clear_text(text):
    pattern = "[{}]".format(string.punctuation)
    text = [word.lower() for word in text]
    text = [[re.sub(pattern, "", word) for word in words.split()] for words in text]
    text = [[word for word in words if len(word)>1] for words in text]    
    text = [' '.join(words) for words in text]
    return np.array(text)

In [21]:
corpus_clear = clear_text(corpus)
print(corpus_clear)

['espresso armazenamento de dados distribuído consistente com linha do tempo orientado documentos do linkedin os clientes que buscam dados de perfil leem um subconjunto de campos de documentos inteiros do espresso em diferentes casos de uso existem três partes principais do espresso roteadores nós de armazenamento gerenciamento de cluster os roteadores recebem solicitações de clientes online encaminham as solicitações aos nós de armazenamento rotas que especificam uri para obter nome do banco de dados procuram na tabela de roteamento os nós de armazenamento apropriados para os quais enviar solicitações ele também reúne os resultados os envia de volta aos clientes um nó de armazenamento armazena fornece documentos fornece suporte transações para documentos com mesma chave de partição mantém log de confirmação para replicação espresso usa apache helix para monitorar manter cluster'
 'você terá um cliente acessando um sistema com acesso várias bases dados terá sistemas que estarão aliment

### 2) Remover stopwords

In [22]:
from nltk.corpus import stopwords
def clear_stopwords(text, language):
    filtered_text = []
    for t in text:
        words = [w for w in t.split() if not w in stopwords.words(language)]
        words = ' '.join(words)
        filtered_text.append(words)
    return np.array(filtered_text)   

In [23]:
import nltk
nltk.download('stopwords')
corpus_filtered = clear_stopwords(corpus_clear, "portuguese")
print(corpus_filtered)

['espresso armazenamento dados distribuído consistente linha tempo orientado documentos linkedin clientes buscam dados perfil leem subconjunto campos documentos inteiros espresso diferentes casos uso existem três partes principais espresso roteadores armazenamento gerenciamento cluster roteadores recebem solicitações clientes online encaminham solicitações armazenamento rotas especificam uri obter nome banco dados procuram tabela roteamento armazenamento apropriados quais enviar solicitações reúne resultados envia volta clientes nó armazenamento armazena fornece documentos fornece suporte transações documentos mesma chave partição mantém log confirmação replicação espresso usa apache helix monitorar manter cluster'
 'cliente acessando sistema acesso várias bases dados sistemas estarão alimentado bases fazendo gestão']


[nltk_data] Downloading package stopwords to /home/jalves/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Usando a biblioteca sklearn

In [24]:
def create_dataframe(matrix, tokens):
    doc_names = [f'text_{i+1}' for i, _ in enumerate(matrix)]
    df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
    return(df)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorize = CountVectorizer()
vector_matrix = vectorize.fit_transform(corpus_filtered)

tokens = vectorize.get_feature_names()
create_dataframe(vector_matrix.toarray(),tokens)



,acessando,acesso,alimentado,apache,apropriados,armazena,armazenamento,banco,bases,buscam,...,suporte,tabela,tempo,transações,três,uri,usa,uso,volta,várias
text_1,0,0,0,1,1,1,5,1,0,1,...,1,1,1,1,1,1,1,1,1,0
text_2,1,1,1,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
cosine_similarity_matrix = cosine_similarity(vector_matrix)
create_dataframe(cosine_similarity_matrix,['text_1','text_2'])

,text_1,text_2
text_1,1.000000,0.063036
text_2,0.063036,1.000000
